In [ ]:
import ROOT as root
import root_pandas as rp
import uproot
import sys
import os
import analysis_variables as a_v
import numpy as np
import pandas as pd
from pylab import *
from array import array
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import probfit as pf
import importlib
import pyarrow
importlib.reload(a_v)

In [ ]:
importlib.reload(a_v)

from multiprocessing import Process, Pool
import multiprocessing.managers
import tqdm

import warnings
warnings.filterwarnings("ignore")

in_Path = a_v.input_dir_extratracks
out_Path = in_Path

channels = ['...', '...', '...', '...', '...', '...','...', '...', '...', '...', '...','...'] 

nomeTree = '...'
sample = '...'

def merger(channel):
    root_file = f"{sample}_{channel}.root"

    print('Reading the root file', root_file)
    if 'offres' in channel:
        df1 = rp.read_root(in_Path + root_file, 
                           key = nomeTree, 
                           where='...<0.9 && ...<0.6 && ...>0.5')
        print('Substituting ... and ...')
        df1['...'] = np.sqrt(a_v.var * a_v.var / 4  - pow(a_v.var / a_v.var,2) * (df1.loc[:,'...']) * (df1.loc[:,'...']))
        df1['...'] = a_v.var / a_v.var * df1.loc[:,'d0_ECMS'] - a_v.var/ 2
        df1 = df1.query('...>5.27 and ... > -0.15 and ... < 0.1')
    else:
        df1 = rp.read_root(in_Path + root_file, 
                           key = nomeTree, 
                           where='...<0.9 && ...<0.6 && ...>0.5 && ... > 5.24 && ... > -0.15 && ... < 0.1')
    print(root_file, '->before dropping duplicates', len(df1))
    df1 = df1.drop_duplicates()
    print(root_file, '->after dropping duplicates', len(df1))
    df1.to_parquet(in_Path + root_file.replace('.root','_skim.parq'), compression='GZIP')
    print(root_file, 'done')

n_cpus = 2

pool = Pool(n_cpus)
    
run_list = channels
pool_result = pool.map(merger, run_list)

pool.close()
pool.join()
